# Importation

In [75]:
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings("ignore")
import sys
import re
import pandas as pd
from PyPDF2 import PdfReader
import cv2
import numpy as np
from PIL import ImageGrab
import time
import os
import pytesseract
from PIL import Image
import fitz
import cv2
import pytesseract
from PIL import Image

In [76]:
from pdf2image import convert_from_path

In [69]:
pip install pyautogui

     ---------------------------------------- 0.0/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------ --------------------------------- 10.2/61.2 kB ? eta -:--:--
     ------------ ------------------------- 20.5/61.2 kB 131.3 kB/s eta 0:00:01
     ------------------- ------------------ 30.7/61.2 kB 130.4 kB/s eta 0:00:01
     ------------------------- ------------ 41.0/61.2 kB 178.6 kB/s eta 0:00:01
     -------------------------------------- 61.2/61.2 kB 217.0 kB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to buil

In [70]:
import pyautogui

# Transforming to pictures

In [12]:
# PDF file paths
pdfPMBOK = "PMBOK6-2017.pdf" #431-494
pdfPMI = "practice-standard-project-risk-management.pdf" #1-116
output_folder_1="PMBOOKimgs"
output_folder_2="Practiceimgs"

In [42]:
def convert_pdf_to_images(pdf_file_path, output_folder, start_page, end_page):
    images = convert_from_path(pdf_file_path, output_folder=output_folder, fmt='png', first_page=start_page, last_page=end_page, output_file="page")

    print("Done.")

In [43]:
convert_pdf_to_images(pdfPMBOK, output_folder_1, 431, 494)

Done.


In [44]:
convert_pdf_to_images(pdfPMI, output_folder_2, 1, 116)

Done.


# Extracting Figures

### YOLO: (does not work)

###### must install yolov3.weights yolov3.cfg coco.names (ask chatgpt)

In [80]:
def yolo_object_detection(image_path):
    # Load YOLO
    net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
    classes = []
    with open("coco.names", "r") as f:
        classes = [line.strip() for line in f.readlines()]
    layer_names = net.getUnconnectedOutLayersNames()

    # Load image
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    # Preprocess image for YOLO
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(layer_names)

    # Get class IDs, confidences, and bounding boxes
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            print(f"Confidence: {confidence}")
            if confidence > 0.1 and class_id == 0:  # Class ID 0 is "person"
                center_x, center_y, w, h = (detection[:4] * np.array([width, height, width, height])).astype(int)
                x, y = int(center_x - w / 2), int(center_y - h / 2)
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, int(w), int(h)])

    # Apply non-maximum suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Save screenshots for detected regions
    print(indices)

    for i in indices:
        i = i[0]
        box = boxes[i]
        x, y, w, h = box

        # Take a screenshot of the detected region using pyautogui
        screenshot = pyautogui.screenshot(region=(x, y, w, h))
        screenshot.save(f"screenshot_{i}.png")

In [81]:
yolo_object_detection("PMBOOKimgs/page0001-432.png")

Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confidence: 0.0
Confiden

### openCV:

In [122]:
def detect_shapes(image_path, min_contour_area=1000):
    # Read the image
    image = cv2.imread("PMBOOKimgs/"+image_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply GaussianBlur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Use Canny edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    k=0
    for contour in contours:
        k=k+1
        # Calculate the contour area
        area = cv2.contourArea(contour)

        # If the contour area is greater than the minimum threshold
        if area >= min_contour_area:
            # Approximate the contour as a polygon
            epsilon = 0.04 * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, True)

            # Check if the polygon has 4 vertices, indicating a rectangle
            if len(approx) == 4:
                # Get the bounding box coordinates of the rectangle
                x, y, w, h = cv2.boundingRect(contour)

                # Define a region of interest (ROI) using the rectangle coordinates
                roi = image[y:y+h, x:x+w]

                # Save the ROI as an image for inspection
                cv2.imwrite("screenshots/"+image_path+"/"+str(k)+"detected__"+image_path, roi)                
                

In [123]:
detect_shapes("page0001-445.png")

In [124]:
for k in os.listdir("PMBOOKimgs/"):
    os.makedirs("screenshots/"+k, exist_ok=True)
    detect_shapes(k)

# Extracting Texts from images

In [25]:
pytesseract.pytesseract.tesseract_cmd = r'C:\Users\hamma\AppData\Local\Tesseract-OCR\tesseract.exe'

In [73]:
def extract_text(image_path):
    # Read the image
    image = cv2.imread(image_path)
    

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Use adaptive thresholding to segment text from the background
    _, threshold = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    threshold = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 1)

    # Remove horizontal and vertical lines using morphological operations
    kernel = np.ones((4, 4), np.uint8)
    clean_image = cv2.morphologyEx(threshold, cv2.MORPH_CLOSE, kernel)
    text = pytesseract.image_to_string(Image.fromarray(clean_image))


    print(text)
    # Save the extracted text to a file
    #with open(output_text_path, 'w', encoding='utf-8') as text_file:
        #text_file.write(text)

In [74]:
extract_text("PMBOOKimgs/page0001-445.png")

11.2 IDENTIFY RISKS

Kdentify Risks Is the process of identifying Individual profect risks as well as sources of overall profect risk, and
documenting thelr characteristics. The key benefit of this process Is the documentation of exdsting indbvidual project risics
and the sources of overall profect risk. It also brings together informrtion so the profect team can respond appropretoty
to identified risks. This process Is performed throughout the prokect. The Inputs, tools and techniques, and outputs of the .
process are depicted in Aigure 11-6. Figure 11-7 depicts the deta flow diagram for the process. rs

AAS)
re—e————v—rv—v—r—

\
Y




# Extracting Texts from pdf

In [ ]:


def extract_text_and_figures(pdf_path, output_text_path, output_figures_path):
    doc = fitz.open(pdf_path)
    text_output = []

    for page_number in range(doc.page_count):
        page = doc[page_number]
        images = page.get_images(full=True)

        for img_index, image in enumerate(images):
            base_image = doc.extract_image(image)
            image_bytes = base_image["image"]
            image_path = f"figure_page{page_number + 1}_img{img_index + 1}.png"

            with open(image_path, "wb") as img_file:
                img_file.write(image_bytes)

            # OCR on the image
            img_cv2 = cv2.imread(image_path)
            gray = cv2.cvtColor(img_cv2, cv2.COLOR_BGR2GRAY)
            _, threshold = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY_INV)
            text = pytesseract.image_to_string(Image.fromarray(threshold))

            # Save the extracted text
            text_output.append(f"Page {page_number + 1}, Figure {img_index + 1}:\n{text}\n")

    # Save the extracted text to a file
    with open(output_text_path, 'w', encoding='utf-8') as text_file:
        text_file.writelines(text_output)

    doc.close()


In [85]:

def extract_plain_text_from_pdf(pdf_file_path):
    # Open the PDF file
    pdf_document = fitz.open(pdf_file_path)

    # Initialize an empty string to store the extracted plain text
    extracted_text = ""

    for page_number in range(431,495):
        page = pdf_document.load_page(page_number)
                # Extraire le texte de la page
        text = page.get_text()

        # Divisez le texte en blocs en utilisant des sauts de ligne ou d'autres séparateurs pertinents
        blocks = text.split("\n")

        # Parcourir les blocs de texte
        for block in blocks:
            # Supprimez les espaces inutiles et les caractères spéciaux
            block = block.strip()

            # Identifiez les figures en fonction de leur contenu ou de leur mise en forme
            if "Figure" in block:
                continue
            # Identifiez les titres en gras
            elif block.isupper():
                extracted_text+=("=== Titre en Gras ===\n")
                extracted_text+=(block + "\n\n")
            # Identifiez les titres normaux
            elif block.istitle():
                extracted_text+=("=== Titre Normal ===\n")
                extracted_text+=(block + "\n\n")
            # Identifiez les éléments de liste (items) en fonction de leur mise en forme
            elif block.startswith("- "):
                continue
            # Les autres blocs sont considérés comme du texte
            else:
                extracted_text+=("=== Texte ===\n")
                extracted_text+=(block + "\n\n")
    # Close the PDF document
    pdf_document.close()

    return extracted_text

# Extract plain text from the PDF
extracted_text = extract_plain_text_from_pdf(pdfPMBOK)

# Print or save the extracted plain text
print(extracted_text)

=== Texte ===
Not For Distribution, Sale or Reproduction.

=== Texte ===
396

=== Titre Normal ===
Part 1 - Guide

=== Titre Normal ===
.1 Inputs

=== Texte ===
.1 Project charter

=== Texte ===
.2 Project management plan

=== Texte ===
.3 Project documents

=== Texte ===
.4  Enterprise environmental

=== Texte ===
factors

=== Texte ===
.5 Organizational process

=== Texte ===
assets

=== Titre Normal ===
.2 Tools & Techniques

=== Texte ===
.1 Expert judgment

=== Texte ===
.2 Data analysis

=== Titre Normal ===
.3 Meetings

=== Titre Normal ===
.3 Outputs

=== Texte ===
.1 Risk management plan

=== Titre Normal ===
Project Risk

=== Titre Normal ===
Management Overview

=== Titre Normal ===
11.1 Plan

=== Titre Normal ===
Risk Management

=== Titre Normal ===
.1 Inputs

=== Texte ===
.1 Project management plan

=== Texte ===
.2 Project documents

=== Titre Normal ===
.3 Agreements

=== Titre Normal ===
.4 Procurement

=== Texte ===


=== Texte ===


=== Texte ===
documentation

=== 